In [246]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import date,timedelta
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import seaborn as sns
import pandas_ta as ta
import numpy as np
import json
from tkinter import *
from tkinter import messagebox, ttk, END
from matplotlib import dates

In [247]:
today = date.today()

d1 = today.strftime('%Y-%m-%d')
end_date = d1
d2 = date.today() - timedelta(days=365)
d2 = d2.strftime('%Y-%m-%d')
start_date = d2

In [248]:
from curl_cffi import requests
session = requests.Session(impersonate="chrome")
ticker = input('Write ticker: ')
stock_data = yf.download(ticker, start = start_date, end = end_date, progress=False, auto_adjust=False, session=session)
if isinstance(stock_data.columns, pd.MultiIndex):
    stock_data.columns = stock_data.columns.droplevel(1)
stock_data

Price,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2025-02-04,2735.051270,2735.051270,2888.250000,2636.165771,2883.818359,48795275985
2025-02-05,2787.779053,2787.779053,2824.400146,2701.104980,2735.225830,31960764447
2025-02-06,2688.400879,2688.400879,2857.142090,2662.445801,2787.658691,29500645692
2025-02-07,2622.211182,2622.211182,2798.028809,2564.968262,2688.897461,29526663418
2025-02-08,2632.308350,2632.308350,2665.484619,2591.778564,2623.001953,17059263074
...,...,...,...,...,...,...
2026-01-30,2702.375244,2702.375244,2823.914551,2633.838867,2818.135498,41877689970
2026-01-31,2445.091797,2445.091797,2709.533691,2248.698730,2702.301514,47569532203
2026-02-01,2267.964844,2267.964844,2472.646240,2224.117920,2445.069336,40358630560


In [249]:
#RSI, Garman, ATR, MACD
stock_data['RSI'] = ta.rsi(close = stock_data["Adj Close"], length=14)

stock_data['Garman'] = ((np.log(stock_data['High'])- np.log(stock_data['Low']))**2)/2-(2*np.log(2)-1)*((np.log(stock_data['Adj Close']) -np.log(stock_data['Open']))**2)

def compute_atr(stock_data):
    atr = ta.atr(high=stock_data['High'], low=stock_data['Low'], close=stock_data['Close'], length=14)
    return atr.sub(atr.mean()).div(atr.std())
stock_data['ATR'] = compute_atr(stock_data)

def compute_macd(stock_data, short_period=12, long_period=26, signal_period=9):
    # Compute MACD, Signal Line, and Histogram using pandas_ta
    macd = ta.macd(
        close=stock_data['Close'], 
        fast=short_period, 
        slow=long_period, 
        signal=signal_period
    )
    
    # Assign the results to new columns in the DataFrame
    stock_data['MACD'] = macd['MACD_12_26_9']  # Default naming format
    stock_data['Signal'] = macd['MACDs_12_26_9']
    stock_data['MACD_Histogram'] = macd['MACDh_12_26_9']
    
    return stock_data

# Apply the function to compute MACD
stock_data = compute_macd(stock_data)

today = date.today() 
two_weeks = today - timedelta(days=14)

#Last 14 days
technical_analysis = stock_data.iloc[-14:]
technical_analysis

Price,Adj Close,Close,High,Low,Open,Volume,RSI,Garman,ATR,MACD,Signal,MACD_Histogram
Date,,,,,,,,,,,,
2026-01-21,2978.901123,2978.901123,3064.390625,2867.264404,2935.882080,35797876195,40.790603,0.002129,-0.906633,14.074398,42.565801,-28.491402
2026-01-22,2949.633301,2949.633301,3034.338135,2905.562744,2979.307861,21993593333,39.424494,0.000902,-0.898506,-2.927225,33.467196,-36.394420
2026-01-23,2953.256348,2953.256348,3013.954102,2892.520996,2949.445801,22417700837,39.693744,0.000845,-0.902992,-15.925208,23.588715,-39.513923
2026-01-24,2948.740723,2948.740723,2965.468750,2942.444580,2953.238281,8198666763,39.458333,0.000029,-1.068434,-26.287550,13.613462,-39.901012
2026-01-25,2815.902344,2815.902344,2955.016846,2785.896729,2948.534424,22414089670,33.217273,0.000918,-0.982632,-44.703415,1.950087,-46.653501
2026-01-26,2926.457031,2926.457031,2945.997559,2811.074951,2815.768799,29668620102,41.509033,0.000525,-0.959002,-49.803181,-8.400567,-41.402614
2026-01-27,3022.210938,3022.210938,3031.038574,2898.311523,2926.201172,27976005223,47.579821,0.000600,-0.940659,-45.592678,-15.838989,-29.753689
2026-01-28,3006.607422,3006.607422,3040.718506,2981.589600,3022.237549,21751566455,46.728702,0.000182,-1.044240,-43.019001,-21.274991,-21.744010
2026-01-29,2818.232666,2818.232666,3008.037598,2751.645020,3006.261719,37487198041,37.911616,0.002357,-0.817141,-55.539389,-28.127871,-27.411518


In [250]:
import plotly.graph_objects as go

# ----------------------------
# Indicators
# ----------------------------
stock_data["M10"] = stock_data["Close"].rolling(window=10).mean()
stock_data["M20"] = stock_data["Close"].rolling(window=20).mean()

stock_data["M40"] = stock_data["Close"].rolling(window=40).mean()
stock_data["M80"] = stock_data["Close"].rolling(window=80).mean()

stock_data["SD"] = stock_data["Close"].rolling(window=20).std()
stock_data["UB"] = stock_data["M20"] + 2 * stock_data["SD"]
stock_data["LB"] = stock_data["M20"] - 2 * stock_data["SD"]


fig = go.Figure()

# 0 Close (line)
fig.add_trace(go.Scatter(
    x=stock_data.index, y=stock_data["Close"],
    mode="lines", name="Close"
))

# 1 Candles
fig.add_trace(go.Candlestick(
    x=stock_data.index,
    open=stock_data["Open"],
    high=stock_data["High"],
    low=stock_data["Low"],
    close=stock_data["Close"],
    name="Candles"
))

# 2-3 MA 10/20
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data["M10"], mode="lines", name="M10"))
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data["M20"], mode="lines", name="M20"))

# 4-5 MA 40/80
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data["M40"], mode="lines", name="M40"))
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data["M80"], mode="lines", name="M80"))

# 6-8 Bollinger Bands
fig.add_trace(go.Scatter(
    x=stock_data.index, y=stock_data["UB"],
    mode="lines", name="Upper BB"
))
fig.add_trace(go.Scatter(
    x=stock_data.index, y=stock_data["LB"],
    mode="lines", name="Lower BB",
    fill="tonexty"  # fill between UB (previous trace) and LB
))
fig.add_trace(go.Scatter(
    x=stock_data.index, y=stock_data["M20"],
    mode="lines", name="Middle BB (M20)"
))

# ----------------------------
# Visibility presets (length MUST be 9)
# ----------------------------
PRICE_CANDLES = [False, True,  False, False, False, False, False, False, False]
PRICE_LINE    = [True,  False, False, False, False, False, False, False, False]

MA_10_20      = [True,  False, True,  True,  False, False, False, False, False]
MA_40_80      = [True,  False, False, False, True,  True,  False, False, False]

BB_VIEW       = [True,  False, False, True,  False, False, True,  True,  True]
# (BB_VIEW includes Close + M20 + UB/LB + Middle; you can tweak if you want)

# Default view
for t in fig.data:
    t.visible = False
fig.data[1].visible = True  # default = candles

# ----------------------------
# Layout + Buttons (positioned to avoid title conflict)
# ----------------------------
fig.update_layout(
    title=dict(text=f"{ticker} Price", x=0.01, y=0.97),  # keep title slightly lower
    xaxis_title="Date",
    yaxis_title="Price",
    plot_bgcolor="lightgrey",
    xaxis_rangeslider_visible=False,
    width=1050,
    height=650,
    margin=dict(t=110, r=30, l=60, b=50),  # extra top margin for buttons + title
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            x=1.0,
            y=1.16,
            xanchor="right",
            yanchor="top",
            pad={"r": 10, "t": 10},
            buttons=[
                dict(
                    label="Candles",
                    method="update",
                    args=[{"visible": PRICE_CANDLES},
                          {"title": f"{ticker} Price (Candles)", "yaxis": {"title": "Price"}}],
                ),
                dict(
                    label="Line",
                    method="update",
                    args=[{"visible": PRICE_LINE},
                          {"title": f"{ticker} Price (Line)", "yaxis": {"title": "Price"}}],
                ),
                dict(
                    label="MA 10/20",
                    method="update",
                    args=[{"visible": MA_10_20},
                          {"title": f"{ticker} Moving Averages (10/20)", "yaxis": {"title": "Price"}}],
                ),
                dict(
                    label="MA 40/80",
                    method="update",
                    args=[{"visible": MA_40_80},
                          {"title": f"{ticker} Moving Averages (40/80)", "yaxis": {"title": "Price"}}],
                ),
                dict(
                    label="Bollinger",
                    method="update",
                    args=[{"visible": BB_VIEW},
                          {"title": f"{ticker} Bollinger Bands (20, 2σ)", "yaxis": {"title": "Price"}}],
                ),
            ],
        )
    ],
)

fig.show()


Sharpe and Sortino Ratio

In [251]:
stock_data = stock_data.copy()
stock_data["Daily Returns"] = stock_data["Adj Close"].pct_change()
stock_data.dropna(inplace=True)

def compute_sharpe_ratio(data, risk_free_rate=0, periods=365):
    """
    Computes daily and annualized Sharpe Ratio
    """
    returns = data["Daily Returns"]
    excess_returns = returns - risk_free_rate / periods

    daily_sharpe = excess_returns.mean() / returns.std()
    annual_sharpe = daily_sharpe * np.sqrt(periods)

    return daily_sharpe, annual_sharpe

def compute_sortino_ratio(data, threshold=0, risk_free_rate=0, periods=365):
    """
    Computes annualized Sortino Ratio
    """
    returns = data["Daily Returns"]
    downside_returns = returns[returns < threshold]

    downside_std = downside_returns.std()
    excess_return = returns.mean() - risk_free_rate / periods

    sortino = (excess_return / downside_std) * np.sqrt(periods)
    return sortino


daily_sharpe, annual_sharpe = compute_sharpe_ratio(stock_data)
sortino = compute_sortino_ratio(stock_data)

print(f"Daily Sharpe Ratio for {ticker}: {daily_sharpe:.4f}")
print(f"Annual Sharpe Ratio for {ticker}: {annual_sharpe:.4f}")
print(f"Sortino Ratio for {ticker}: {sortino:.4f}")


Daily Sharpe Ratio for ETH-USD: 0.0384
Annual Sharpe Ratio for ETH-USD: 0.7344
Sortino Ratio for ETH-USD: 1.2011


In [252]:
prices = pd.concat([xrp, btc, eth], axis=1)
prices.columns = ["XRP", "BTC", "ETH"]
prices = prices.dropna()

# daily log returns
log_returns = np.log(prices / prices.shift(1)).dropna()

def monte_carlo_paths(price_series, mu, sigma, days=365, n_sims=5000, seed=42):
    """
    Geometric Brownian Motion (GBM) simulation using log returns
    """
    rng = np.random.default_rng(seed)
    s0 = float(price_series.iloc[-1])

    # random daily shocks
    z = rng.standard_normal((days, n_sims))

    # GBM in log space: S(t) = S0 * exp( (mu - 0.5*sigma^2)*t + sigma*W )
    drift = (mu - 0.5 * sigma**2)
    daily_log_steps = drift + sigma * z
    log_paths = np.cumsum(daily_log_steps, axis=0)
    paths = s0 * np.exp(log_paths)

    # add S0 as day 0
    paths = np.vstack([np.full((1, n_sims), s0), paths])
    return paths  # shape: (days+1, n_sims)

def summarize_paths(paths):
    terminal = paths[-1, :]
    s0 = paths[0, 0]
    summary = {
        "Start": s0,
        "Mean End": terminal.mean(),
        "Median End": np.median(terminal),
        "P05 End": np.percentile(terminal, 5),
        "P95 End": np.percentile(terminal, 95),
        "Prob End > Start": (terminal > s0).mean()
    }
    return summary, terminal

DAYS = 365
N_SIMS = 5000

results = {}
terminals = {}

for coin in ["XRP", "BTC", "ETH"]:
    mu = log_returns[coin].mean()
    sigma = log_returns[coin].std()

    paths = monte_carlo_paths(prices[coin], mu, sigma, days=DAYS, n_sims=N_SIMS, seed=42)
    summary, terminal = summarize_paths(paths)

    results[coin] = summary
    terminals[coin] = terminal

# Print results
for coin, s in results.items():
    print(f"\n--- {coin} Monte Carlo ({DAYS} days, {N_SIMS} sims) ---")
    print(f"Start:            {s['Start']:.6f}")
    print(f"Mean End:         {s['Mean End']:.6f}")
    print(f"Median End:       {s['Median End']:.6f}")
    print(f"5th % End:        {s['P05 End']:.6f}")
    print(f"95th % End:       {s['P95 End']:.6f}")
    print(f"Prob End > Start: {s['Prob End > Start']*100:.2f}%")


coin_list = ["XRP", "BTC", "ETH"]

PLOT_SIMS = 200
plot_paths = {}

for coin in coin_list:
    mu = log_returns[coin].mean()
    sigma = log_returns[coin].std()
    plot_paths[coin] = monte_carlo_paths(prices[coin], mu, sigma, days=DAYS, n_sims=PLOT_SIMS, seed=7)

fig = go.Figure()

trace_groups = {}

for i, coin in enumerate(coin_list):
    paths = plot_paths[coin]
    trace_idxs = []

    # spaghetti paths
    for s in range(paths.shape[1]):
        fig.add_trace(go.Scatter(
            x=np.arange(paths.shape[0]),
            y=paths[:, s],
            mode="lines",
            name=f"{coin} sim",
            line=dict(width=1),
            opacity=0.15,
            showlegend=False,
            visible=(i == 0)
        ))
        trace_idxs.append(len(fig.data) - 1)

    # mean path
    fig.add_trace(go.Scatter(
        x=np.arange(paths.shape[0]),
        y=paths.mean(axis=1),
        mode="lines",
        name=f"{coin} mean",
        line=dict(width=3),
        visible=(i == 0)
    ))
    trace_idxs.append(len(fig.data) - 1)

    trace_groups[coin] = trace_idxs

buttons = []
for coin in coin_list:
    visible = [False] * len(fig.data)
    for idx in trace_groups[coin]:
        visible[idx] = True

    buttons.append(dict(
        label=coin,
        method="update",
        args=[
            {"visible": visible},
            {"title": f"{coin} Monte Carlo Paths ({DAYS} days, {PLOT_SIMS} sims shown)",
             "xaxis": {"title": "Day"},
             "yaxis": {"title": "Simulated Price"}}
        ],
    ))

fig.update_layout(
    title=f"XRP Monte Carlo Paths ({DAYS} days, {PLOT_SIMS} sims shown)",
    width=1050,
    height=600,
    plot_bgcolor="lightgrey",
    updatemenus=[dict(
        type="buttons",
        direction="right",
        x=1.0,
        y=1.15,
        xanchor="right",
        yanchor="top",
        pad={"r": 10, "t": 10},
        buttons=buttons
    )],
    margin=dict(t=110, r=30, l=60, b=50)
)

fig.show()

fig2 = go.Figure()

hist_groups = {}
for i, coin in enumerate(coin_list):
    terminal = terminals[coin]
    fig2.add_trace(go.Histogram(
        x=terminal,
        nbinsx=60,
        name=f"{coin} terminal",
        opacity=0.75,
        visible=(i == 0)
    ))
    hist_groups[coin] = len(fig2.data) - 1

buttons2 = []
for coin in coin_list:
    visible = [False] * len(fig2.data)
    visible[hist_groups[coin]] = True

    s = results[coin]
    subtitle = (f"Start={s['Start']:.6f} | Mean={s['Mean End']:.6f} | "
                f"P05={s['P05 End']:.6f} | P95={s['P95 End']:.6f} | "
                f"P(End>Start)={s['Prob End > Start']*100:.2f}%")

    buttons2.append(dict(
        label=coin,
        method="update",
        args=[
            {"visible": visible},
            {"title": f"{coin} Terminal Price Distribution<br><sup>{subtitle}</sup>",
             "xaxis": {"title": "Terminal Price"},
             "yaxis": {"title": "Count"}}
        ],
    ))

fig2.update_layout(
    title="XRP Terminal Price Distribution",
    width=1050,
    height=600,
    plot_bgcolor="lightgrey",
    updatemenus=[dict(
        type="buttons",
        direction="right",
        x=1.0,
        y=1.15,
        xanchor="right",
        yanchor="top",
        pad={"r": 10, "t": 10},
        buttons=buttons2
    )],
    margin=dict(t=110, r=30, l=60, b=50)
)

fig2.show()



--- XRP Monte Carlo (365 days, 5000 sims) ---
Start:            1.574199
Mean End:         0.989494
Median End:       0.718369
5th % End:        0.198648
95th % End:       2.666190
Prob End > Start: 16.72%

--- BTC Monte Carlo (365 days, 5000 sims) ---
Start:            75633.546875
Mean End:         58784.834298
Median End:       53698.998078
5th % End:        27229.897304
95th % End:       107360.865720
Prob End > Start: 21.16%

--- ETH Monte Carlo (365 days, 5000 sims) ---
Start:            2227.558838
Mean End:         1836.554282
Median End:       1376.984015
5th % End:        407.124712
95th % End:       4773.360414
Prob End > Start: 26.60%
